In [30]:
from __future__ import print_function

%matplotlib inline
from awips.dataaccess import DataAccessLayer

import matplotlib.tri as mtri
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np

from metpy.calc import get_wind_components, lcl, dry_lapse, parcel_profile
from metpy.plots import SkewT, Hodograph
from metpy.units import units, concatenate

plt.rcParams['figure.figsize'] = (12, 14)

# Set EDEX host
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest()

# Data type bufrua
request.setDatatype("bufrua")
# Parameters
request.setParameters("tpMan","tdMan","prMan","htMan","wdMan","wsMan", "staName")

dir(request)
# Station ID (name doesn't work yet)
#request.setLocationNames("KDNR")
#datatimes = DataAccessLayer.getAvailableTimes(request)

# Get most recent record
#response = DataAccessLayer.getGeometryData(request,times=datatimes[-1].validPeriod)

['_DefaultDataRequest__makeLevel',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__metaclass__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 'addIdentifier',
 'datatype',
 'envelope',
 'getDatatype',
 'getEnvelope',
 'getIdentifiers',
 'getLevels',
 'getLocationNames',
 'getParameters',
 'identifiers',
 'levels',
 'locationNames',
 'parameters',
 'removeIdentifier',
 'setDatatype',
 'setEnvelope',
 'setLevels',
 'setLocationNames',
 'setParameters']

In [23]:
print(response)

for ob in response:
    print(ob)

ob.getString("staName")

[<awips.dataaccess.PyGeometryData.PyGeometryData object at 0x0000000007B18A20>, <awips.dataaccess.PyGeometryData.PyGeometryData object at 0x0000000007B18BA8>, <awips.dataaccess.PyGeometryData.PyGeometryData object at 0x0000000003C0D5C0>, <awips.dataaccess.PyGeometryData.PyGeometryData object at 0x0000000003C0D5F8>, <awips.dataaccess.PyGeometryData.PyGeometryData object at 0x0000000003C0D588>, <awips.dataaccess.PyGeometryData.PyGeometryData object at 0x0000000003C0D668>, <awips.dataaccess.PyGeometryData.PyGeometryData object at 0x0000000003C0D6A0>, <awips.dataaccess.PyGeometryData.PyGeometryData object at 0x0000000003C0D6D8>, <awips.dataaccess.PyGeometryData.PyGeometryData object at 0x0000000003C0D710>, <awips.dataaccess.PyGeometryData.PyGeometryData object at 0x0000000003C0D748>, <awips.dataaccess.PyGeometryData.PyGeometryData object at 0x0000000003C0D780>, <awips.dataaccess.PyGeometryData.PyGeometryData object at 0x0000000003C0D7B8>, <awips.dataaccess.PyGeometryData.PyGeometryData obj

'KDNR'

In [ ]:
# Initialize data arrays
tpMan,tdMan,prMan,htMan,wdMan,wsMan = [],[],[],[],[],[]

# Build ordered arrays
for ob in response:
    print float(ob.getString("prMan")), float(ob.getString("wsMan"))
    tpMan.append(float(ob.getString("tpMan")))
    tdMan.append(float(ob.getString("tdMan")))
    prMan.append(float(ob.getString("prMan")))
    htMan.append(float(ob.getString("htMan")))
    wdMan.append(float(ob.getString("wdMan")))
    wsMan.append(float(ob.getString("wsMan")))

# we can use units.* here...
T = np.array(tpMan)-273.15
Td = np.array(tdMan)-273.15
p = np.array(prMan)/100
height = np.array(htMan)
direc = np.array(wdMan)
spd = np.array(wsMan)
u, v = get_wind_components(spd, np.deg2rad(direc))

p = p * units.mbar
T = T * units.degC
Td = Td * units.degC
spd = spd * units.knot
direc = direc * units.deg

# Create a skewT plot
skew = SkewT()

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, T, 'r')
skew.plot(p, Td, 'g')
skew.plot_barbs(p, u, v)
skew.ax.set_ylim(1000, 100)
skew.ax.set_xlim(-40, 60)

# Calculate LCL height and plot as black dot
l = lcl(p[0], T[0], Td[0])
lcl_temp = dry_lapse(concatenate((p[0], l)), T[0])[-1].to('degC')
skew.plot(l, lcl_temp, 'ko', markerfacecolor='black')

# Calculate full parcel profile and add to plot as black line
prof = parcel_profile(p, T[0], Td[0]).to('degC')
skew.plot(p, prof, 'k', linewidth=2)

# Example of coloring area between profiles
skew.ax.fill_betweenx(p, T, prof, where=T>=prof, facecolor='blue', alpha=0.4)
skew.ax.fill_betweenx(p, T, prof, where=T<prof, facecolor='red', alpha=0.4)

# An example of a slanted line at constant T -- in this case the 0 isotherm
l = skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

# Draw hodograph
ax_hod = inset_axes(skew.ax, '40%', '40%', loc=3)
h = Hodograph(ax_hod, component_range=80.)
h.add_grid(increment=20)
h.plot_colormapped(u, v, spd)

# Show the plot
plt.show()